In [1]:
import nhl 
import requests
import pandas as pd
from sqlalchemy import create_engine
from credentials import HOCKEY_DB_NAME, HOCKEY_DB_PWD
from datetime import date, timedelta
import time
from pytz import timezone
from IPython.display import clear_output

from schedule import parse_schedule, get_schedule
from play import Shot, Goal, Penalty, Faceoff, Other, process_game, process_play_bundle
from plotnine import *

from betting_utils import convert_prob_to_american

pd.set_option('display.max_columns', None)

import numpy as np

In [2]:
tmp = requests.get('https://moneypuck.com/moneypuck/playerData/careers/gameByGame/all_teams.csv')
open("all_teams.csv", "wb").write(tmp.content)

99078393

In [3]:
dat = pd.read_csv('all_teams.csv')
dat = dat[dat['playoffGame'] == 0]

print(dat['gameDate'].max())

20230109


In [4]:
dat['gameDate'] = pd.to_datetime(dat['gameDate'], format = '%Y%m%d')

full_names = ['Tampa Bay Lightning', 'New York Rangers', 'Washington Capitals', 'Minnesota Wild', 
              'Pittsburgh Penguins', 'Chicago Blackhawks', 'Dallas Stars', 'Edmonton Oilers', 
              'Calgary Flames', 'Arizona Coyotes', 'Vancouver Canucks', 'San Jose Sharks',
              'Toronto Maple Leafs', 'Buffalo Sabres', 'Ottawa Senators', 'Detroit Red Wings',
              'Winnipeg Jets', 'St. Louis Blues', 'Colorado Avalanche', 'Carolina Hurricanes',
              'Columbus Blue Jackets', 'Montréal Canadiens', 'Los Angeles Kings', 'Anaheim Ducks', 
              'New York Islanders', 'Boston Bruins', 'New Jersey Devils', 'Philadelphia Flyers', 
              'Seattle Kraken', 'Florida Panthers', 'Nashville Predators', 'Vegas Golden Knights',
              'New Jersey Devils', 'Tampa Bay Lightning', 'San Jose Sharks', 'Los Angeles Kings']

abbrev_names =  ['TBL', 'NYR', 'WSH', 'MIN', 
                 'PIT', 'CHI', 'DAL', 'EDM', 
                 'CGY', 'ARI', 'VAN', 'SJS',
                 'TOR', 'BUF', 'OTT', 'DET', 
                 'WPG', 'STL', 'COL', 'CAR', 
                 'CBJ', 'MTL', 'LAK', 'ANA',
                 'NYI', 'BOS', 'NJD', 'PHI', 
                 'SEA', 'FLA', 'NSH', 'VGK',
                 'N.J', 'T.B', 'S.J', 'L.A']

dat['playerTeam'] = dat['playerTeam'].replace(abbrev_names, full_names)
dat['opposingTeam'] = dat['opposingTeam'].replace(abbrev_names, full_names)
dat['team'] = dat['team'].replace(abbrev_names, full_names)
dat['name'] = dat['name'].replace(abbrev_names, full_names)
dat['game_final'] = True

In [5]:
#s = get_schedule(startdate = date.today() + 1, enddate = date.today() + 1)
s = get_schedule(startdate = date.today(), enddate = date.today() + timedelta(days = 1))

s['game_date'] = pd.to_datetime(s['game_date'])
s['game_date'] = s['game_date'].apply(lambda x: x.astimezone(timezone('US/Eastern')))
s['game_date'] = s['game_date'].apply(lambda x: x.tz_localize(None))
#s['game_date'] = s['game_date'].apply(lambda x: str(x.strftime('%Y%m%d')))

s = s.melt(id_vars = ['game_pk', 'game_date'], value_vars = ['home_team', 'away_team'])

s['variable'] = s['variable'].replace(['home_team', 'away_team'], ['HOME', 'AWAY'])

s['season'] = 2022
s['team'] = s['value']
s['name'] = s['value']
s['playerTeam'] = s['value']
s['gameId'] = s['game_pk']
s['home_or_away'] = s['variable']
s['gameDate'] = s['game_date']
s['game_final'] = False
s['position'] = 'Team Level'

s['situation'] = 'all'

s2 = s.copy()

for situation in ['5on5', '5on4', '4on5']:
    s3 = s2.copy()
    s3['situation'] = situation
    s = s.append(s3)
    
s

C:\Users\Chuck\AppData\Local\Temp\ipykernel_11068\1447300304.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
C:\Users\Chuck\AppData\Local\Temp\ipykernel_11068\1447300304.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.


,game_pk,game_date,variable,value,season,team,name,playerTeam,gameId,home_or_away,gameDate,game_final,position,situation
0,2022020648,2023-01-10 19:00:00,HOME,Tampa Bay Lightning,2022,Tampa Bay Lightning,Tampa Bay Lightning,Tampa Bay Lightning,2022020648,HOME,2023-01-10 19:00:00,False,Team Level,all
1,2022020649,2023-01-10 19:00:00,HOME,New York Rangers,2022,New York Rangers,New York Rangers,New York Rangers,2022020649,HOME,2023-01-10 19:00:00,False,Team Level,all
2,2022020650,2023-01-10 19:00:00,HOME,Carolina Hurricanes,2022,Carolina Hurricanes,Carolina Hurricanes,Carolina Hurricanes,2022020650,HOME,2023-01-10 19:00:00,False,Team Level,all
3,2022020651,2023-01-10 19:00:00,HOME,Buffalo Sabres,2022,Buffalo Sabres,Buffalo Sabres,Buffalo Sabres,2022020651,HOME,2023-01-10 19:00:00,False,Team Level,all
4,2022020652,2023-01-10 19:00:00,HOME,Pittsburgh Penguins,2022,Pittsburgh Penguins,Pittsburgh Penguins,Pittsburgh Penguins,2022020652,HOME,2023-01-10 19:00:00,False,Team Level,all
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23,2022020656,2023-01-10 21:30:00,AWAY,Florida Panthers,2022,Florida Panthers,Florida Panthers,Florida Panthers,2022020656,AWAY,2023-01-10 21:30:00,False,Team Level,4on5
24,2022020659,2023-01-11 19:00:00,AWAY,Washington Capitals,2022,Washington Capitals,Washington Capitals,Washington Capitals,2022020659,AWAY,2023-01-11 19:00:00,False,Team Level,4on5
25,2022020658,2023-01-11 19:30:00,AWAY,Nashville Predators,2022,Nashville Predators,Nashville Predators,Nashville Predators,2022020658,AWAY,2023-01-11 19:30:00,False,Team Level,4on5
26,2022020660,2023-01-11 22:00:00,AWAY,Edmonton Oilers,2022,Edmonton Oilers,Edmonton Oilers,Edmonton Oilers,2022020660,AWAY,2023-01-11 22:00:00,False,Team Level,4on5


In [6]:
dat = pd.concat([dat, s[['team', 'season', 'name', 'gameId', 'playerTeam', 'home_or_away', 'game_final', 'gameDate', 'position', 'situation']]]).reset_index()

#dat = dat.fillna(-1)

del dat['index']


### Even strength
Expected goals from non-rebound shots (Even Strength Adjusted) - Each shot the team has taken at even strength is given a probability that it will be a goal. This variable is the sum of all the team's shot probabilities divided by the sum of all shot probabilities against them. Unlike corsi or fenwick, this variable factors in shot quality. This variable is the most influential variable in the model. Rebound shots were excluded since they were shown to be less predictive of future wins than just counting non-rebound shots. This is likely due to a higher degree of luck in getting rebounds than other shots, and thus less repeatability.


Unblocked Shot Attempts For % (Even Strength Adjusted) -This variable, also known as score adjusted fenwick, describes the team's share of unblocked shot attempts at even strength in their previous games. The count of unblocked shot attempts is adjusted for the score in game at time that they occurred. For example, teams generally take more unblocked shots attempts when trailing by a goal, so shot attempts in that situation are discounted by about 9%. Unblocked shot attempts include shots on goals and shots that miss the net. In the older version of the model that did not included expected goals, score adjusted fenwick was the most important variable. Like all the other variables below, this variable is weighted to give more weight to more recent games in the season.

Shooting Percentage (Even Strength) -The team's shooting percentage in even strength situations.

In [7]:
even_strength = dat.query('situation == "5on5"')

even_strength['xgoals_non_rebound_for'] = even_strength['xGoalsFor'] - even_strength['xGoalsFromActualReboundsOfShotsFor']
even_strength['xgoals_non_rebound_against'] = even_strength['xGoalsAgainst'] - even_strength['xGoalsFromActualReboundsOfShotsAgainst']
even_strength['xgoals_non_rebound_ratio'] = even_strength['xgoals_non_rebound_for'] / even_strength['xgoals_non_rebound_against']

even_strength['unblocked_shot_attempts_ratio'] = even_strength['scoreAdjustedUnblockedShotAttemptsFor'] / even_strength['scoreAdjustedUnblockedShotAttemptsAgainst']

even_strength['shooting_percentage'] = even_strength['goalsFor'] / even_strength['shotsOnGoalFor']

even_strength['blocked_shot_attempts_ratio'] = even_strength['blockedShotAttemptsFor'] / (even_strength['blockedShotAttemptsAgainst'] + even_strength['blockedShotAttemptsFor'])


even_strength = even_strength[['team', 'home_or_away', 'gameId', 'gameDate', 'xgoals_non_rebound_for', 'xgoals_non_rebound_against', 'xgoals_non_rebound_ratio', 'scoreAdjustedUnblockedShotAttemptsFor', 'scoreAdjustedUnblockedShotAttemptsAgainst', 'unblocked_shot_attempts_ratio', 'blockedShotAttemptsFor', 'blockedShotAttemptsAgainst', 'blocked_shot_attempts_ratio', 'shooting_percentage']]

C:\Users\Chuck\AppData\Local\Temp\ipykernel_11068\2575282439.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Chuck\AppData\Local\Temp\ipykernel_11068\2575282439.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Chuck\AppData\Local\Temp\ipykernel_11068\2575282439.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-doc

### Power play
Expected goals in power play and penalty kill situations - Similar to the expected goal variable above, but only looking at man-advantage situations. The team's expected goals on 5 on 4 power plays is divided by the total amount of power play time they've had. The same is calculated for the total expected goals against them on penalty kills divided by the amount of time the team has spent killing penalties. The final variable is the power play expected goals metric divided by the penalty kill expected goals metric.

Share of Power Play Time -The team's % share of power play time. (Compared to the amount of time they spend on the penalty kill)

In [8]:
import re
for c in dat.columns:
    if 'Against' in c:
        tmp = dat.query('situation == "all"').copy()
        tmp['win'] = tmp['goalsFor'] > tmp['goalsAgainst']
        tmp['metric_win'] = tmp[re.sub(string = c, pattern = 'Against', repl = 'For')] > tmp[c]
        
        print(re.sub(string = c, pattern = 'Against', repl = 'For'))
        print(tmp.groupby('metric_win')['win'].mean())

xOnGoalFor
metric_win
False    0.450859
True     0.441930
Name: win, dtype: float64
xGoalsFor
metric_win
False    0.304983
True     0.588163
Name: win, dtype: float64
xReboundsFor
metric_win
False    0.445063
True     0.447759
Name: win, dtype: float64
xFreezeFor
metric_win
False    0.467038
True     0.425659
Name: win, dtype: float64
xPlayStoppedFor
metric_win
False    0.457616
True     0.435111
Name: win, dtype: float64
xPlayContinuedInZoneFor
metric_win
False    0.478139
True     0.414506
Name: win, dtype: float64
xPlayContinuedOutsideZoneFor
metric_win
False    0.463256
True     0.429464
Name: win, dtype: float64
flurryAdjustedxGoalsFor
metric_win
False    0.296725
True     0.596423
Name: win, dtype: float64
scoreVenueAdjustedxGoalsFor
metric_win
False    0.280347
True     0.612837
Name: win, dtype: float64
flurryScoreVenueAdjustedxGoalsFor
metric_win
False    0.272055
True     0.621168
Name: win, dtype: float64
shotsOnGoalFor
metric_win
False    0.434065
True     0.459755
Name: wi

In [9]:
pp = dat.query('situation == "5on4"')
pp['xgoals_pp_for'] = (pp['xGoalsFor']) / pp['iceTime']
pp.loc[np.isnan(pp['xgoals_pp_for']), 'xgoals_pp_for'] = 0
pp['ice_time_pp_for'] = pp['iceTime']

pk = dat.query('situation == "4on5"')
pk['xgoals_pk_against'] = (pk['xGoalsAgainst']) / pk['iceTime']
pk.loc[np.isnan(pk['xgoals_pk_against']), 'xgoals_pk_against'] = 0
pk['ice_time_pk_against'] = pk['iceTime']

penalty = pp[['team', 'home_or_away', 'gameId', 'gameDate', 'xgoals_pp_for', 'ice_time_pp_for']].merge(pk[['team', 'gameId', 'gameDate', 'xgoals_pk_against', 'ice_time_pk_against']])
penalty['pp_xgoal_ratio'] = penalty['xgoals_pp_for']  / (penalty['xgoals_pk_against'] + penalty['xgoals_pp_for'])
penalty['pp_time_ratio'] = penalty['ice_time_pp_for'] / (penalty['ice_time_pk_against'] + penalty['ice_time_pp_for'])
penalty

C:\Users\Chuck\AppData\Local\Temp\ipykernel_11068\1126977308.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Chuck\AppData\Local\Temp\ipykernel_11068\1126977308.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Chuck\AppData\Local\Temp\ipykernel_11068\1126977308.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-doc

,team,home_or_away,gameId,gameDate,xgoals_pp_for,ice_time_pp_for,xgoals_pk_against,ice_time_pk_against,pp_xgoal_ratio,pp_time_ratio
0,New York Rangers,AWAY,2008020001,2008-10-04 00:00:00,0.000693,807.0,0.001434,468.0,0.325749,0.632941
1,New York Rangers,HOME,2008020003,2008-10-05 00:00:00,0.003244,160.0,0.000052,523.0,0.984334,0.234261
2,New York Rangers,HOME,2008020010,2008-10-10 00:00:00,0.000728,360.0,0.000064,360.0,0.919298,0.500000
3,New York Rangers,AWAY,2008020019,2008-10-11 00:00:00,0.003658,269.0,0.002463,231.0,0.597594,0.538000
4,New York Rangers,HOME,2008020034,2008-10-13 00:00:00,0.002628,188.0,0.000922,360.0,0.740210,0.343066
...,...,...,...,...,...,...,...,...,...,...
34029,Florida Panthers,AWAY,2022020656,2023-01-10 21:30:00,0.000000,NaN,0.000000,NaN,NaN,NaN
34030,Washington Capitals,AWAY,2022020659,2023-01-11 19:00:00,0.000000,NaN,0.000000,NaN,NaN,NaN
34031,Nashville Predators,AWAY,2022020658,2023-01-11 19:30:00,0.000000,NaN,0.000000,NaN,NaN,NaN
34032,Edmonton Oilers,AWAY,2022020660,2023-01-11 22:00:00,0.000000,NaN,0.000000,NaN,NaN,NaN


### All situations

In [10]:
all_sit = dat.query('situation == "all"')
all_sit['save_pct_for'] = 1 - (all_sit['goalsAgainst'] / (all_sit['shotsOnGoalAgainst']))

all_sit['last_game_date'] = all_sit.groupby('team')['gameDate'].shift(1)
import datetime as dt
all_sit['is_b2b'] = 1.0 * (all_sit['gameDate'] - all_sit['last_game_date']) < dt.timedelta(hours = 36)

all_sit = all_sit[['team', 'home_or_away', 'gameId', 'gameDate', 'goalsAgainst', 'shotsOnGoalAgainst', 'save_pct_for', 'is_b2b']]
all_sit

C:\Users\Chuck\AppData\Local\Temp\ipykernel_11068\3934945935.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Chuck\AppData\Local\Temp\ipykernel_11068\3934945935.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Chuck\AppData\Local\Temp\ipykernel_11068\3934945935.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-doc

,team,home_or_away,gameId,gameDate,goalsAgainst,shotsOnGoalAgainst,save_pct_for,is_b2b
1,New York Rangers,AWAY,2008020001,2008-10-04 00:00:00,1.0,21.0,0.952381,False
6,New York Rangers,HOME,2008020003,2008-10-05 00:00:00,1.0,19.0,0.947368,True
11,New York Rangers,HOME,2008020010,2008-10-10 00:00:00,2.0,32.0,0.937500,False
16,New York Rangers,AWAY,2008020019,2008-10-11 00:00:00,3.0,28.0,0.892857,True
21,New York Rangers,HOME,2008020034,2008-10-13 00:00:00,1.0,27.0,0.962963,False
...,...,...,...,...,...,...,...,...
170053,Florida Panthers,AWAY,2022020656,2023-01-10 21:30:00,NaN,NaN,NaN,False
170054,Washington Capitals,AWAY,2022020659,2023-01-11 19:00:00,NaN,NaN,NaN,False
170055,Nashville Predators,AWAY,2022020658,2023-01-11 19:30:00,NaN,NaN,NaN,False
170056,Edmonton Oilers,AWAY,2022020660,2023-01-11 22:00:00,NaN,NaN,NaN,False


### Put it together

In [11]:
targets = dat.query('situation == "all"')[['season', 'team', 'home_or_away', 'game_final', 'gameId', 'gameDate', 'goalsFor', 'goalsAgainst', 'shotsOnGoalFor', 'shotsOnGoalAgainst']]
targets['team_wins'] = 1.0 * (targets['goalsFor'] > targets['goalsAgainst'])
targets['team_saves'] = (targets['shotsOnGoalAgainst'] - targets['goalsAgainst'])

In [12]:
targets = targets.merge(even_strength)
targets = targets.merge(penalty)
targets = targets.merge(all_sit)

In [13]:
targets = targets.sort_values(['gameDate', 'gameId', 'home_or_away'])

In [14]:
targets

,season,team,home_or_away,game_final,gameId,gameDate,goalsFor,goalsAgainst,shotsOnGoalFor,shotsOnGoalAgainst,team_wins,team_saves,xgoals_non_rebound_for,xgoals_non_rebound_against,xgoals_non_rebound_ratio,scoreAdjustedUnblockedShotAttemptsFor,scoreAdjustedUnblockedShotAttemptsAgainst,unblocked_shot_attempts_ratio,blockedShotAttemptsFor,blockedShotAttemptsAgainst,blocked_shot_attempts_ratio,shooting_percentage,xgoals_pp_for,ice_time_pp_for,xgoals_pk_against,ice_time_pk_against,pp_xgoal_ratio,pp_time_ratio,save_pct_for,is_b2b
0,2008,New York Rangers,AWAY,True,2008020001,2008-10-04 00:00:00,2.0,1.0,41.0,21.0,1.0,20.0,1.693,1.625,1.041846,36.697,19.369,1.894625,10.0,5.0,0.666667,0.035714,0.000693,807.0,0.001434,468.0,0.325749,0.632941,0.952381,False
7568,2008,Tampa Bay Lightning,HOME,True,2008020001,2008-10-04 00:00:00,1.0,2.0,21.0,41.0,0.0,39.0,1.625,1.693,0.959835,19.369,36.697,0.527809,5.0,10.0,0.333333,0.090909,0.001434,468.0,0.000693,807.0,0.674251,0.367059,0.951220,False
1240,2008,Pittsburgh Penguins,AWAY,True,2008020002,2008-10-04 00:00:00,4.0,3.0,30.0,35.0,1.0,32.0,1.148,1.243,0.923572,27.352,26.987,1.013525,12.0,5.0,0.705882,0.117647,0.001041,705.0,0.002039,311.0,0.338062,0.693898,0.914286,False
20380,2008,Ottawa Senators,HOME,True,2008020002,2008-10-04 00:00:00,3.0,4.0,35.0,30.0,0.0,26.0,1.243,1.148,1.082753,26.987,27.352,0.986655,5.0,12.0,0.294118,0.045455,0.002039,311.0,0.001041,705.0,0.661938,0.306102,0.866667,False
7569,2008,Tampa Bay Lightning,AWAY,True,2008020003,2008-10-05 00:00:00,1.0,2.0,19.0,39.0,0.0,37.0,0.803,1.093,0.734675,22.984,36.211,0.634724,9.0,14.0,0.391304,0.076923,0.000052,523.0,0.003244,160.0,0.015666,0.765739,0.948718,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34017,2022,Toronto Maple Leafs,HOME,False,2022020658,2023-01-11 19:30:00,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,0.000000,NaN,NaN,NaN,NaN,False
34032,2022,Edmonton Oilers,AWAY,False,2022020660,2023-01-11 22:00:00,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,0.000000,NaN,NaN,NaN,NaN,False
34018,2022,Anaheim Ducks,HOME,False,2022020660,2023-01-11 22:00:00,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,0.000000,NaN,NaN,NaN,NaN,False
34033,2022,San Jose Sharks,AWAY,False,2022020661,2023-01-11 22:00:00,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,0.000000,NaN,NaN,NaN,NaN,True


In [15]:
targets['xgoals_non_rebound_for_l50'] = targets.groupby(['team'])['xgoals_non_rebound_for'].apply(lambda x: x.shift().rolling(50, min_periods = 25).apply(np.nanmean))
targets['xgoals_non_rebound_against_l50'] = targets.groupby(['team'])['xgoals_non_rebound_against'].apply(lambda x: x.shift().rolling(50, min_periods = 25).apply(np.nanmean))

targets['score_adjusted_unblocked_shot_attempts_for_l50'] = targets.groupby(['team'])['scoreAdjustedUnblockedShotAttemptsFor'].apply(lambda x: x.shift().rolling(50, min_periods = 25).apply(np.nanmean))
targets['score_adjusted_unblocked_shot_attempts_against_l50'] = targets.groupby(['team'])['scoreAdjustedUnblockedShotAttemptsAgainst'].apply(lambda x: x.shift().rolling(50, min_periods = 25).apply(np.nanmean))

targets['blocked_shot_attempts_for_l50'] = targets.groupby(['team'])['blockedShotAttemptsFor'].apply(lambda x: x.shift().rolling(50, min_periods = 25).apply(np.nanmean))
targets['blocked_shot_attempts_against_l50'] = targets.groupby(['team'])['blockedShotAttemptsAgainst'].apply(lambda x: x.shift().rolling(50, min_periods = 25).apply(np.nanmean))

targets['shooting_percentage_l50'] = targets.groupby(['team'])['shooting_percentage'].apply(lambda x: x.shift().rolling(50, min_periods = 25).apply(np.nanmean))

targets['xgoals_pp_for_l50'] = targets.groupby(['team'])['xgoals_pp_for'].apply(lambda x: x.shift().rolling(50, min_periods = 25).apply(np.nanmean))
targets['xgoals_pk_against_l50'] = targets.groupby(['team'])['xgoals_pk_against'].apply(lambda x: x.shift().rolling(50, min_periods = 25).apply(np.nanmean))
targets['ice_time_pp_for_l50'] = targets.groupby(['team'])['ice_time_pp_for'].apply(lambda x: x.shift().rolling(50, min_periods = 25).apply(np.nanmean))
targets['ice_time_pk_against_l50'] = targets.groupby(['team'])['ice_time_pk_against'].apply(lambda x: x.shift().rolling(50, min_periods = 25).apply(np.nanmean))

targets['save_pct_for_l50'] = targets.groupby(['team'])['save_pct_for'].apply(lambda x: x.shift().rolling(50, min_periods = 25).apply(np.nanmean))

targets['xgoals_non_rebound_ratio_l50'] = targets['xgoals_non_rebound_for_l50'] / (targets['xgoals_non_rebound_for_l50'] + targets['xgoals_non_rebound_against_l50'])
targets['score_adjusted_unblocked_shot_attempts_ratio_l50'] = targets['score_adjusted_unblocked_shot_attempts_for_l50'] / (targets['score_adjusted_unblocked_shot_attempts_for_l50'] + targets['score_adjusted_unblocked_shot_attempts_against_l50'])
targets['pp_xgoal_ratio_l50'] = targets['xgoals_pp_for_l50'] / (targets['xgoals_pk_against_l50'] + targets['xgoals_pp_for_l50'])
targets['pp_time_ratio_l50'] = targets['ice_time_pp_for_l50'] / (targets['ice_time_pp_for_l50'] + targets['ice_time_pk_against_l50'])
targets['blocked_shot_attempts_ratio_l50'] = targets['blocked_shot_attempts_for_l50'] / (targets['blocked_shot_attempts_for_l50'] + targets['blocked_shot_attempts_against_l50'])


In [16]:
from sklearn.metrics import log_loss, accuracy_score
import statsmodels.formula.api as smf

In [17]:
targets = targets[~pd.isnull(targets['xgoals_non_rebound_for_l50'])]
targets['game_tie'] = targets['goalsFor'] == targets['goalsAgainst']

C:\Users\Chuck\AppData\Local\Temp\ipykernel_11068\1969556888.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [18]:
m_win = smf.logit('team_wins ~ xgoals_non_rebound_for_l50 + xgoals_non_rebound_against_l50 + xgoals_non_rebound_ratio_l50 +\
score_adjusted_unblocked_shot_attempts_for_l50 + score_adjusted_unblocked_shot_attempts_against_l50 + score_adjusted_unblocked_shot_attempts_ratio_l50 +\
shooting_percentage_l50 + xgoals_pp_for_l50 + xgoals_pk_against_l50 + ice_time_pp_for_l50 + ice_time_pk_against_l50 +\
pp_xgoal_ratio_l50 + pp_time_ratio_l50 + save_pct_for_l50 +\
is_b2b', data = targets.query('game_final == True & game_tie == False'))

g_win = m_win.fit()#_regularized(alpha = 2, L1_wt = 1)

Optimization terminated successfully.
         Current function value: 0.681691
         Iterations 6


In [19]:
g_win.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:              team_wins   No. Observations:                29674
Model:                          Logit   Df Residuals:                    29658
Method:                           MLE   Df Model:                           15
Date:                Tue, 10 Jan 2023   Pseudo R-squ.:                 0.01653
Time:                        09:18:35   Log-Likelihood:                -20229.
converged:                       True   LL-Null:                       -20568.
Covariance Type:            nonrobust   LLR p-value:                3.596e-135
======================================================================================================================
                                                         coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------------------
Intercept                                             -4.6770      4.171     -1.121      0.262     -12.851       3.497
is_b2b[T.True]                                        -0.2520      0.031     -8.068      0.000      -0.313      -0.191
xgoals_non_rebound_for_l50                             0.8279      1.078      0.768      0.442      -1.284       2.940
xgoals_non_rebound_against_l50                        -0.4455      1.089     -0.409      0.682      -2.579       1.688
xgoals_non_rebound_ratio_l50                          -1.4682      6.658     -0.221      0.825     -14.517      11.580
score_adjusted_unblocked_shot_attempts_for_l50         0.0311      0.074      0.419      0.675      -0.114       0.176
score_adjusted_unblocked_shot_attempts_against_l50    -0.0691      0.075     -0.926      0.354      -0.215       0.077
score_adjusted_unblocked_shot_attempts_ratio_l50      -1.0958      9.662     -0.113      0.910     -20.033      17.842
shooting_percentage_l50                                7.2676      1.240      5.860      0.000       4.837       9.698
xgoals_pp_for_l50                                     -6.4267     26.701     -0.241      0.810     -58.760      45.906
xgoals_pk_against_l50                                 27.8852     25.710      1.085      0.278     -22.504      78.275
ice_time_pp_for_l50                                    0.0012      0.003      0.454      0.650      -0.004       0.007
ice_time_pk_against_l50                               -0.0016      0.003     -0.591      0.554      -0.007       0.004
pp_xgoal_ratio_l50                                     0.4974      0.296      1.678      0.093      -0.083       1.078
pp_time_ratio_l50                                     -1.9518      3.285     -0.594      0.552      -8.390       4.486
save_pct_for_l50                                       7.5845      1.195      6.345      0.000       5.242       9.927
======================================================================================================================
"""

In [20]:
m_gf = smf.ols('goalsFor ~ xgoals_non_rebound_for_l50 + xgoals_non_rebound_against_l50 + xgoals_non_rebound_ratio_l50 +\
score_adjusted_unblocked_shot_attempts_for_l50 + score_adjusted_unblocked_shot_attempts_against_l50 + score_adjusted_unblocked_shot_attempts_ratio_l50 +\
shooting_percentage_l50 + xgoals_pp_for_l50 + xgoals_pk_against_l50 + ice_time_pp_for_l50 + ice_time_pk_against_l50 +\
pp_xgoal_ratio_l50 + pp_time_ratio_l50 + save_pct_for_l50 +\
is_b2b', data = targets.query('game_final == True & game_tie == False'))

g_gf = m_gf.fit()#_regularized(alpha = 2, L1_wt = 1)

In [21]:
m_ga = smf.ols('goalsAgainst ~ xgoals_non_rebound_for_l50 + xgoals_non_rebound_against_l50 + xgoals_non_rebound_ratio_l50 +\
score_adjusted_unblocked_shot_attempts_for_l50 + score_adjusted_unblocked_shot_attempts_against_l50 + score_adjusted_unblocked_shot_attempts_ratio_l50 +\
shooting_percentage_l50 + xgoals_pp_for_l50 + xgoals_pk_against_l50 + ice_time_pp_for_l50 + ice_time_pk_against_l50 +\
pp_xgoal_ratio_l50 + pp_time_ratio_l50 + save_pct_for_l50 +\
is_b2b', data = targets.query('game_final == True & game_tie == False'))

g_ga = m_ga.fit()#_regularized(alpha = 2, L1_wt = 1)

In [22]:
targets['pred_win'] = g_win.predict(targets)
targets['pred_gf'] = g_gf.predict(targets)
targets['pred_ga'] = g_ga.predict(targets)

C:\Users\Chuck\AppData\Local\Temp\ipykernel_11068\1106702066.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Chuck\AppData\Local\Temp\ipykernel_11068\1106702066.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Chuck\AppData\Local\Temp\ipykernel_11068\1106702066.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-doc

In [23]:
final = targets.pivot_table(index = ['gameId', 'gameDate', 'season', 'game_final', 'game_tie'], columns = 'home_or_away', values = ['team_wins', 'goalsFor', 'pred_win', 'pred_gf', 'pred_ga'])

final.columns = ['_'.join(col).strip() for col in final.columns.values]

final = final.reset_index()

final = final.merge(targets.query('home_or_away == "HOME"')[['gameId', 'gameDate', 'season', 'game_final', 'team']])

In [24]:
final

,gameId,gameDate,season,game_final,game_tie,goalsFor_AWAY,goalsFor_HOME,pred_ga_AWAY,pred_ga_HOME,pred_gf_AWAY,pred_gf_HOME,pred_win_AWAY,pred_win_HOME,team_wins_AWAY,team_wins_HOME,team
0,2008020346,2008-11-30 00:00:00,2008,True,False,NaN,0.0,NaN,2.459418,NaN,2.753278,NaN,0.544891,NaN,0.0,New York Rangers
1,2008020362,2008-12-03 00:00:00,2008,True,True,NaN,2.0,NaN,2.494873,NaN,2.709343,NaN,0.528951,NaN,0.0,New York Rangers
2,2008020368,2008-12-04 00:00:00,2008,True,False,NaN,5.0,NaN,2.808381,NaN,2.980508,NaN,0.536914,NaN,1.0,Washington Capitals
3,2008020369,2008-12-04 00:00:00,2008,True,False,NaN,2.0,NaN,2.930159,NaN,2.815485,NaN,0.472814,NaN,0.0,Carolina Hurricanes
4,2008020375,2008-12-04 00:00:00,2008,True,False,NaN,3.0,NaN,2.289011,NaN,3.261478,NaN,0.690099,NaN,1.0,San Jose Sharks
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16495,2022020658,2023-01-11 19:30:00,2022,False,False,NaN,NaN,3.126290,2.728518,3.098961,3.400764,0.492859,0.618422,0.0,0.0,Toronto Maple Leafs
16496,2022020659,2023-01-11 19:00:00,2022,False,False,NaN,NaN,2.822296,3.229616,3.148527,2.789133,0.556096,0.424529,0.0,0.0,Philadelphia Flyers
16497,2022020660,2023-01-11 22:00:00,2022,False,False,NaN,NaN,3.073653,3.740355,3.267135,2.489377,0.528701,0.289460,0.0,0.0,Anaheim Ducks
16498,2022020661,2023-01-11 22:00:00,2022,False,False,NaN,NaN,3.385198,2.859256,2.959957,3.157895,0.436975,0.558728,0.0,0.0,Los Angeles Kings


In [25]:
m_win = smf.logit('team_wins_HOME ~ pred_win_AWAY + pred_win_HOME', data = final.query('game_final == True & game_tie == False'))

g_win = m_win.fit()#_regularized(alpha = 2, L1_wt = 1)

final['pred_win_HOME'] = g_win.predict(final)
final['pred_win_AWAY'] = 1 - final['pred_win_HOME']

Optimization terminated successfully.
         Current function value: 0.666659
         Iterations 5


In [26]:
g_win.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:         team_wins_HOME   No. Observations:                14703
Model:                          Logit   Df Residuals:                    14700
Method:                           MLE   Df Model:                            2
Date:                Tue, 10 Jan 2023   Pseudo R-squ.:                 0.03123
Time:                        09:18:36   Log-Likelihood:                -9801.9
converged:                       True   LL-Null:                       -10118.
Covariance Type:            nonrobust   LLR p-value:                6.113e-138
=================================================================================
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept         0.0898      0.161      0.556      0.578      -0.226       0.406
pred_win_AWAY    -3.9189      0.228    -17.190      0.000      -4.366      -3.472
pred_win_HOME     4.0942      0.231     17.696      0.000       3.641       4.548
=================================================================================
"""

In [27]:
final['amer_odds_HOME'] = final['pred_win_HOME'].apply(convert_prob_to_american)
final['amer_odds_AWAY'] = final['pred_win_AWAY'].apply(convert_prob_to_american)

In [28]:
m_gf = smf.ols('goalsFor_HOME ~ pred_gf_HOME + pred_ga_AWAY', data = final.query('game_final == True & game_tie == False'))

g_gf = m_gf.fit()#_regularized(alpha = 2, L1_wt = 1)

final['pred_gf_HOME'] = g_gf.predict(final)


In [29]:
g_gf.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          goalsFor_HOME   R-squared:                       0.031
Model:                            OLS   Adj. R-squared:                  0.031
Method:                 Least Squares   F-statistic:                     236.4
Date:                Tue, 10 Jan 2023   Prob (F-statistic):          8.56e-102
Time:                        09:18:36   Log-Likelihood:                -28884.
No. Observations:               14703   AIC:                         5.777e+04
Df Residuals:                   14700   BIC:                         5.780e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       -1.6681      0.217     -7.704      0.000      -2.093      -1.244
pred_gf_HOME     0.8105      0.058     14.029      0.000       0.697       0.924
pred_ga_AWAY     0.8171      0.056     14.696      0.000       0.708       0.926
==============================================================================
Omnibus:                      316.262   Durbin-Watson:                   1.998
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              333.669
Skew:                           0.363   Prob(JB):                     3.50e-73
Kurtosis:                       2.865   Cond. No.                         65.9
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [30]:
m_ga = smf.ols('goalsFor_AWAY ~ pred_ga_HOME + pred_gf_HOME', data = final.query('game_final == True & game_tie == False'))

g_ga = m_ga.fit()#_regularized(alpha = 2, L1_wt = 1)

final['pred_ga_HOME'] = g_ga.predict(final)


In [31]:
final['total'] = final['goalsFor_AWAY'] + final['goalsFor_HOME']

m_tot = smf.ols('total ~ pred_ga_HOME + pred_gf_HOME', data = final.query('game_final == True & game_tie == False'))

g_tot = m_tot.fit()#_regularized(alpha = 2, L1_wt = 1)

final['pred_total'] = g_tot.predict(final)

In [32]:
final.query('game_final == False')[['gameId', 'gameDate', 'team', 'amer_odds_HOME', 'pred_gf_HOME', 'pred_ga_HOME']]

,gameId,gameDate,team,amer_odds_HOME,pred_gf_HOME,pred_ga_HOME
16485,2022020648,2023-01-10 19:00:00,Tampa Bay Lightning,-304.507870,3.934265,2.556424
16486,2022020649,2023-01-10 19:00:00,New York Rangers,-126.998095,3.121405,2.587384
16487,2022020650,2023-01-10 19:00:00,Carolina Hurricanes,-151.154470,3.390895,2.202994
16488,2022020651,2023-01-10 19:00:00,Buffalo Sabres,107.708350,3.333649,2.907119
16489,2022020652,2023-01-10 19:00:00,Pittsburgh Penguins,-188.868372,3.725721,2.643660
16490,2022020653,2023-01-10 19:30:00,New York Islanders,111.289049,3.025345,2.910278
16491,2022020654,2023-01-10 19:00:00,Detroit Red Wings,158.196716,2.803799,3.140171
16492,2022020655,2023-01-10 20:00:00,St. Louis Blues,153.104124,2.987123,3.264631
16493,2022020656,2023-01-10 21:30:00,Colorado Avalanche,-106.841574,3.253790,2.653852
16494,2022020657,2023-01-10 21:00:00,Arizona Coyotes,146.095097,3.121498,3.368238


In [33]:
final.query('game_final == False')[['gameId', 'gameDate', 'team', 'pred_total']].sort_values('pred_total')

,gameId,gameDate,team,pred_total
16487,2022020650,2023-01-10 19:00:00,Carolina Hurricanes,5.672186
16486,2022020649,2023-01-10 19:00:00,New York Rangers,5.729016
16491,2022020654,2023-01-10 19:00:00,Detroit Red Wings,5.876480
16490,2022020653,2023-01-10 19:30:00,New York Islanders,5.899232
16493,2022020656,2023-01-10 21:30:00,Colorado Avalanche,5.907063
16496,2022020659,2023-01-11 19:00:00,Philadelphia Flyers,5.944432
16495,2022020658,2023-01-11 19:30:00,Toronto Maple Leafs,6.123079
16492,2022020655,2023-01-10 20:00:00,St. Louis Blues,6.149187
16488,2022020651,2023-01-10 19:00:00,Buffalo Sabres,6.186445
16498,2022020661,2023-01-11 22:00:00,Los Angeles Kings,6.240303


In [34]:
final[~pd.isnull(final['pred_win_HOME'])].groupby('season').apply(lambda x: log_loss(x['team_wins_HOME'], x['pred_win_HOME']))

season
2008    0.682628
2009    0.698163
2010    0.696032
2011    0.694890
2012    0.688928
2013    0.689443
2014    0.682634
2015    0.689716
2016    0.678602
2017    0.682501
2018    0.679819
2019    0.687875
2020    0.663157
2021    0.651208
2022    0.670044
dtype: float64

In [35]:
final[~pd.isnull(final['pred_win_HOME'])].groupby('season').apply(lambda x: accuracy_score(x['team_wins_HOME'], x['pred_win_HOME'] >= 0.5))

season
2008    0.567696
2009    0.533767
2010    0.543902
2011    0.537339
2012    0.544540
2013    0.549201
2014    0.556098
2015    0.551220
2016    0.564228
2017    0.555377
2018    0.574351
2019    0.550832
2020    0.589862
2021    0.618493
2022    0.596651
dtype: float64

In [36]:
import pandas as pd

In [37]:
dat = pd.read_excel('C:/Users/Chuck/Downloads/nhl odds 2021-22.xlsx')

In [38]:
dat['Rot'] = dat['Rot'].apply(lambda x: (x - 1) - ((x - 1) % 2))

dat['GameID'] = dat.apply(lambda x: '{}_{}'.format(x['Date'], x['Rot']), axis = 1)

In [39]:
tmp = pd.pivot_table(index = ['Date', 'GameID', 'Rot'], columns = 'VH', data = dat, values = ['Final', 'Open', 'Close'])

tmp.columns = ['_'.join(col).strip() for col in tmp.columns.values]

tmp = tmp.reset_index()


In [40]:
tmp['home_wins'] = 1.0 * (tmp['Final_H'] > tmp['Final_V'])

In [41]:
def american_odds_to_prob(a):
    if a < 0:
        a = -a
        return (a / (a + 100))
    else:
        return (100 / (a + 100))

In [42]:
tmp['pred_win_HOME_open'] = tmp['Open_H'].apply(american_odds_to_prob)
tmp['pred_win_HOME_close'] = tmp['Close_H'].apply(american_odds_to_prob)

In [43]:
tmp = tmp[~pd.isnull(tmp['Close_H'])]

In [44]:
accuracy_score(tmp['home_wins'], tmp['pred_win_HOME_open'] >= 0.5)

0.6381156316916489

In [45]:
accuracy_score(tmp['home_wins'], tmp['pred_win_HOME_close'] >= 0.5)

0.6466809421841542

In [46]:
log_loss(tmp['home_wins'], tmp['pred_win_HOME_open'])

0.6498100172638417

In [47]:
log_loss(tmp['home_wins'], tmp['pred_win_HOME_close'])

0.643343933820647

In [48]:
dat['Date'].unique()

array([1012, 1013, 1014, 1015, 1016, 1017, 1018, 1019, 1020, 1021, 1022,
       1023, 1024, 1025, 1026, 1027, 1028, 1029, 1030, 1031, 1101, 1102,
       1103, 1104, 1105, 1106, 1107, 1108, 1109, 1110, 1111, 1112, 1113,
       1114, 1115, 1116, 1117, 1118, 1119, 1120, 1121, 1122, 1123, 1124,
       1126, 1127, 1128, 1129, 1130, 1201, 1202, 1203, 1204, 1205, 1206,
       1207, 1208, 1209, 1210, 1211, 1212, 1214, 1215, 1216, 1217, 1218,
       1219, 1220, 1221, 1228, 1229, 1230, 1231,  101,  102,  103,  104,
        105,  106,  107,  108,  109,  110,  111,  112,  113,  114,  115,
        116,  117,  118,  119,  120,  121,  122,  123,  124,  125,  126,
        127,  128,  129,  130,  131,  201,  202,  207,  208,  209,  210,
        211,  212,  213,  214,  215,  216,  217,  218,  219,  220,  221,
        222,  223,  224,  225,  226,  227,  228,  301,  302,  303,  304,
        305,  306,  307,  308,  309,  310,  311,  312,  313,  314,  315,
        316,  317,  318,  319,  320,  321,  322,  3